In [2]:
consumer_key='68qlYXjh8SQWY4n7tjkxoYHhP'
consumer_secret='7NTFQF19OyUffhwvPa3w6PflEJLtUxEemD1RJjtnyvM85hGs17'
access_token_key='337359768-psJwlyY2MfMt746MrLuiZG87hrGwfxt71yYm092W'
access_token_secret='KMbkwAH8UFBoJOvL22BAhFbH0tHG3waMdFDtcRUnthTTY'

In [3]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json
import datetime as dt
import time
import os
import sys
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
 
api = tweepy.API(auth)

we can read our own timeline (i.e. our Twitter homepage) with:

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    print(status.text) 

In [ ]:
#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):
    def __init__(self):
        self.end_time = time.time() + 600 #in seconds
        
    def on_data(self, data):
        if time.time() > self.end_time:
            sys.exit()
        else:
            try:
                with open('python.json', 'a') as f:
                    f.write(data)
                    return True
            except BaseException as e:
                print("Error on_data: %s" % str(e))
            return True

    def on_error(self, status):
        print(status)


if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token_key, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords 
    stream.filter(track=['python', 'javascript', 'ruby'])

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
tweets_data_path = 'python.json'

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [ ]:
print(len(tweets_data))

In [ ]:
import pandas as pd

In [ ]:
tweets = pd.DataFrame()

In [ ]:
tweets['text'] = list(map(lambda tweet:tweet['text'],tweets_data))
tweets['lang'] = list(map(lambda tweet: tweet['lang'], tweets_data))
tweets['country'] = list(map(lambda tweet: tweet['place']['country'] if tweet['place'] != None else None, tweets_data))

In [ ]:
tweets.head()

In [ ]:
%matplotlib inline

In [ ]:
tweets_by_lang = tweets['lang'].value_counts()

fig, ax = plt.subplots()
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlabel('Languages', fontsize=15)
ax.set_ylabel('Number of tweets' , fontsize=15)
ax.set_title('Top 5 languages', fontsize=15, fontweight='bold')
tweets_by_lang[:5].plot(ax=ax, kind='bar', color='red')


In [ ]:
tweets_by_country = tweets['country'].value_counts()

fig, ax = plt.subplots()
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlabel('Countries', fontsize=15)
ax.set_ylabel('Number of tweets' , fontsize=15)
ax.set_title('Top 5 countries', fontsize=15, fontweight='bold')
tweets_by_country[:5].plot(ax=ax, kind='bar', color='blue')


In [ ]:
import re

In [ ]:
def word_in_text(word, text):
    word = word.lower()
    text = text.lower()
    match = re.search(word, text)
    if match:
        return True
    return False

In [ ]:
tweets['python'] = tweets['text'].apply(lambda tweet: word_in_text('python', tweet))
tweets['javascript'] = tweets['text'].apply(lambda tweet: word_in_text('javascript', tweet))
tweets['ruby'] = tweets['text'].apply(lambda tweet: word_in_text('ruby', tweet))

In [ ]:
print(tweets['python'].value_counts()[True])
print(tweets['javascript'].value_counts()[True])
print(tweets['ruby'].value_counts()[True])

In [ ]:
prg_langs = ['python', 'javascript', 'ruby']
tweets_by_prg_lang = [tweets['python'].value_counts()[True], tweets['javascript'].value_counts()[True], tweets['ruby'].value_counts()[True]]

x_pos = list(range(len(prg_langs)))
width = 0.8
fig, ax = plt.subplots()
plt.bar(x_pos, tweets_by_prg_lang, width, alpha=1, color='g')

# Setting axis labels and ticks
ax.set_ylabel('Number of tweets', fontsize=15)
ax.set_title('Ranking: python vs. javascript vs. ruby (Raw data)', fontsize=10, fontweight='bold')
ax.set_xticks([p + 0.4 * width for p in x_pos])
ax.set_xticklabels(prg_langs)
plt.grid()

In [ ]:
tweets['programming'] = tweets['text'].apply(lambda tweet: word_in_text('programming', tweet))
tweets['tutorial'] = tweets['text'].apply(lambda tweet: word_in_text('tutorial', tweet))
tweets['relevant'] = tweets['text'].apply(lambda tweet: word_in_text('programming', tweet) or word_in_text('tutorial', tweet))

In [ ]:
print(tweets['programming'].value_counts()[True])
print(tweets['tutorial'].value_counts()[True])
print(tweets['relevant'].value_counts()[True])

In [ ]:
print(tweets[tweets['relevant'] == True]['python'].value_counts()[True])
print(tweets[tweets['relevant'] == True]['javascript'].value_counts()[True])
print(list(tweets[tweets['relevant'] == True]['ruby'].value_counts())[0])

In [ ]:
tweets_by_prg_lang = [tweets[tweets['relevant'] == True]['python'].value_counts()[True], 
                      tweets[tweets['relevant'] == True]['javascript'].value_counts()[True], 
                      list(tweets[tweets['relevant'] == True]['ruby'].value_counts())[0]]
x_pos = list(range(len(prg_langs)))
width = 0.8
fig, ax = plt.subplots()
plt.bar(x_pos, tweets_by_prg_lang, width,alpha=1,color='g')
ax.set_ylabel('Number of tweets', fontsize=15)
ax.set_title('Ranking: python vs. javascript vs. ruby (Relevant data)', fontsize=10, fontweight='bold')
ax.set_xticks([p + 0.4 * width for p in x_pos])
ax.set_xticklabels(prg_langs)
plt.grid()

# Extracting links from the relevants tweets

In [ ]:
def extract_link(text):
    regex = r'https?://[^\s<>"]+|www\.[^\s<>"]+'
    match = re.search(regex, text)
    if match:
        return match.group()
    return ''

In [ ]:
tweets['link'] = tweets['text'].apply(lambda tweet: extract_link(tweet))

In [ ]:
tweets_relevant = tweets[tweets['relevant'] == True]
tweets_relevant_with_link = tweets_relevant[tweets_relevant['link'] != '']

In [ ]:
print(tweets_relevant_with_link[tweets_relevant_with_link['python'] == True]['link'])
print(tweets_relevant_with_link[tweets_relevant_with_link['javascript'] == True]['link'])
print(tweets_relevant_with_link[tweets_relevant_with_link['ruby'] == True]['link'])

# Get tweets from one user

In [4]:
#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
consumer_key = consumer_key
consumer_secret = consumer_secret
access_key = access_token_key
access_secret = access_token_secret


def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]

    #write the csv	
    with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)

    pass


if __name__ == '__main__':
    #pass in the username of the account you want to download
    get_all_tweets("adowa_conan")# @names
    print('finished')

getting tweets before 202146255294562303
...400 tweets downloaded so far
getting tweets before 142599445538811903
...600 tweets downloaded so far
getting tweets before 106874128690200577
...718 tweets downloaded so far
getting tweets before 92972361661419519
...718 tweets downloaded so far
finished


In [5]:
#Twitter API credentials
consumer_key = consumer_key
consumer_secret = consumer_secret
access_key = access_token_key
access_secret = access_token_secret


def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))
    
    #transform the tweepy tweets into a 2D array that will populate the csv	
    with open('%s_tweets.json' % screen_name, 'w') as f:
        json.dump([status._json for status in alltweets],f)
    

    pass
    #return alltweets

if __name__ == '__main__':
    #pass in the username of the account you want to download
    get_all_tweets("adowa_conan")# @names
    print('finished')

getting tweets before 202146255294562303
...400 tweets downloaded so far
getting tweets before 142599445538811903
...600 tweets downloaded so far
getting tweets before 106874128690200577
...718 tweets downloaded so far
getting tweets before 92972361661419519
...718 tweets downloaded so far
finished


In [ ]:
import pandas as pd
screen_name = 'realDonaldTrump'
get_all_tweets(screen_name)
print('finished')
tweets_data_path = '%s_tweets.json' % screen_name

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:

    tweet = json.loads(line)
    tweets_data.append(tweet)


tweets_data=tweets_data[0]

In [18]:
tweets=pd.DataFrame()
tweets['text'] = list(map(lambda tweet:tweet['text'],tweets_data))
tweets['time'] = list(map(lambda tweet:tweet['created_at'],tweets_data))
tweets['source']=list(map(lambda tweet:tweet['source'],tweets_data))
tweets.head()

,text,time,source
0,Where was all the outrage from Democrats and t...,Mon Jan 30 14:23:49 +0000 2017,"<a href=""http://twitter.com/download/iphone"" r..."
1,I have made my decision on who I will nominate...,Mon Jan 30 13:43:25 +0000 2017,"<a href=""http://twitter.com/download/android"" ..."
2,If the ban were announced with a one week noti...,Mon Jan 30 13:31:00 +0000 2017,"<a href=""http://twitter.com/download/android"" ..."
3,There is nothing nice about searching for terr...,Mon Jan 30 12:27:05 +0000 2017,"<a href=""http://twitter.com/download/android"" ..."
4,protesters and the tears of Senator Schumer. S...,Mon Jan 30 12:20:49 +0000 2017,"<a href=""http://twitter.com/download/android"" ..."


In [20]:
tweets.time[0].split(' ')

['Mon', 'Jan', '30', '14:23:49', '+0000', '2017']

In [1]:
import json

In [4]:
import pandas as pd
screen_name = 'realDonaldTrump'
with open('%s_tweets.json' % screen_name, 'w') as f:
    tweets_data = json.load(f)

IOError: File not open for reading

ValueError: I/O operation on closed file